In [1]:
# ── Jupyter settings ──────────────────────────────────────────────────────────
%load_ext autoreload
%autoreload 2
import asyncio

In [2]:

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend
from pylabrobot.resources.hamilton import (
    STARLetDeck, MFX_CAR_L5_base, TIP_CAR_480_A00
)
from pylabrobot.resources.hamilton.mfx_modules import MFX_DWP_module_188042
from pylabrobot.resources import (
    HTF,   # 2.2 mL source plate
    Cor_96_wellplate_360ul_Fb       # 360 µL destination plate
)

###############################################################################
# 0) build LH + deck
###############################################################################
backend = STARBackend()
lh      = LiquidHandler(backend=backend, deck=STARLetDeck())
await lh.setup(skip_autoload=True)                      # ← no autoload = faster dev




In [3]:
from typing import Optional

from pylabrobot.resources.height_volume_functions import (
  compute_height_from_volume_rectangle,
  compute_volume_from_height_rectangle,
)
from pylabrobot.resources.plate import Lid, Plate
from pylabrobot.resources.utils import create_ordered_items_2d
from pylabrobot.resources.well import (
  CrossSectionType,
  Well,
  WellBottomType,
)

def BioER_96_wellplate_Vb_2200ul(name: str, lid: Optional[Lid] = None) -> Plate:
  """
  BioER plate similar to NEST and other KingFisher-compatible deep well plates
  https://en.bioer.com/uploadfiles/2024/05/20240513165756879.pdf
  Cat no. BSH06M1T-A
  """
  INNER_WELL_WIDTH = 8.2  # from spec  
  INNER_WELL_LENGTH = 8.2  # from spec

  well_kwargs = {
    "size_x": INNER_WELL_WIDTH,  # measured
    "size_y": INNER_WELL_LENGTH,  # measured
    "size_z": 42.4,  # from spec
    "bottom_type": WellBottomType.V,
    "cross_section_type": CrossSectionType.RECTANGLE,
    "compute_height_from_volume": lambda liquid_volume: compute_height_from_volume_rectangle(
      liquid_volume,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    "compute_volume_from_height": lambda liquid_height: compute_volume_from_height_rectangle(
      liquid_height,
      INNER_WELL_LENGTH,
      INNER_WELL_WIDTH,
    ),
    # "material_z_thickness": 1,
    "material_z_thickness": 0.80, # measured
  }

  return Plate(
    name=name,
    size_x=127.30,  # from spec
    size_y=85.20,  # from spec
    size_z=44.2,  # from spec
    lid=lid,
    model=BioER_96_wellplate_Vb_2200ul.__name__,
    ordered_items=create_ordered_items_2d(
      Well,
      num_items_x=12,
      num_items_y=8,
      dx=9.2,  # measured
      dy=9.5,  # measured
      dz=1.2,  # from spec
      item_dx=9, # from spec
      item_dy=9, # from spec
      **well_kwargs,
    ),
  )

In [4]:
###############################################################################
# # 1) carriers & labware
# ###############################################################################
# tip carrier on rail 1
tip_car = TIP_CAR_480_A00("tip_car")
tip_car[3] = HTF(name="htf_tips")                       # filtered 300 µL tips
lh.deck.assign_child_resource(tip_car, rails=1)

# deep-well source carrier on rail 13
dwp_mod_src  = MFX_DWP_module_188042("dwp_mod_src")
flex_car_src = MFX_CAR_L5_base("flex_car_src", modules={0: dwp_mod_src})
lh.deck.assign_child_resource(flex_car_src, rails=13)

src_plate = BioER_96_wellplate_Vb_2200ul("source_plate")
dwp_mod_src.assign_child_resource(src_plate, location=0)

# destination (flat-bottom) carrier on rail 19
dwp_mod_dst  = MFX_DWP_module_188042("dwp_mod_dst")
flex_car_dst = MFX_CAR_L5_base("flex_car_dst", modules={0: dwp_mod_dst})
lh.deck.assign_child_resource(flex_car_dst, rails=19)

dst_plate = Cor_96_wellplate_360ul_Fb("dest_plate")
dwp_mod_dst.assign_child_resource(dst_plate, location=0)

###############################################################################
# 2) liquid-class & LLD settings for highest accuracy
###############################################################################
precise = dict(
    aspirate_flow_rate=10,           # µL s⁻¹
    dispense_flow_rate=10,
    end_delay_seconds=0.5,     # µL s-¹ (asp, disp)
    settling_time=[0.5]*8        # seconds
)

lld_surface = dict(                       # pressure LLD on every aspirate
    lld_mode=[STARBackend.LLDMode.GAMMA]*8,
    # lld_search_height=[30]*8,             # start 30 mm above deck
    # immersion_depth=[1]*8,                # stop 1 mm below surface
    # immersion_depth_direction=[0]*8
)

lld_surface_follow = lld_surface | {
    # stay 1 mm under the surface the whole time
    "surface_following_distance":[10]*8    
}

# helper to unwrap PLR-generated single-well lists
def first_well(item):
    return item[0] if isinstance(item, (list, tuple)) else item

# row ordering for an 8-channel head
ROWS = "ABCDEFGH"
USE_CHANS = list(range(8))

##############################################################################
# 3) pick up tips & one-shot pre-wet (30 µL water → back to src)
##############################################################################
tip_rack = lh.deck.get_resource("htf_tips")
await lh.pick_up_tips(tip_rack["A3:H3"], use_channels=list(range(8)))

water_src_wells = [first_well(src_plate[f"{row}12"]) for row in ROWS]  # col 12 water

await lh.aspirate(water_src_wells, vols=[300]*8, use_channels=USE_CHANS, **precise, **lld_surface_follow)
# await lh.aspirate(water_src_wells, vols=[150]*8, use_channels=USE_CHANS, lld_mode=[STARBackend.LLDMode.GAMMA]*8, surface_following_distance=[10]*8, liquid_height=[32]*8)
await lh.dispense(water_src_wells, vols=[300]*8, use_channels=USE_CHANS, **precise, blow_out=[1]*8, liquid_height=[32]*8)   # blow-out at the end of pre-wet
# await lh.dispense(water_src_wells, vols=[150]*8, use_channels=USE_CHANS, blow_out=[1]*8, liquid_height=[35]*8)   # blow-out at the end of pre-wet

###############################################################################
# 4) WATER ACCURACY TESTS
###############################################################################
# mapping: (target columns, volume µL, repetitions per well)
water_tests = [
    (range(1, 5),   190, 1),   # cols 1-4, 95 µL twice  → 190 µL total
    (range(5, 9),   150, 1),   # cols 5-8, 75 µL twice  → 150 µL total
    (range(9, 12), 100, 1),   # cols 9-11, 100 µL once → 100 µL total
]

for cols, vol, reps in water_tests:
    for col in cols:
        dst_wells = [first_well(dst_plate[f"{row}{col}"]) for row in ROWS]
        for _ in range(reps):
            try:
                await lh.aspirate(water_src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, **lld_surface_follow)
            except:
                await lh.aspirate(water_src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[1]*8)
        await lh.dispense(dst_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[3]*8)      # no blow-out per spec
            # await lh.dispense(dst_wells,      vols=[vol]*8, use_channels=USE_CHANS, liquid_height=[3]*8)      # no blow-out per spec

# ###############################################################################
# # 5) discard tips, re-rack fresh tips for Orange G
# ###############################################################################
await lh.discard_tips()
await lh.pick_up_tips(tip_rack["A4:H4"], use_channels=list(range(8)))

###############################################################################
# 6) ORANGE G ADD-BACK
###############################################################################
dye_src_wells = [first_well(src_plate[f"{row}1"]) for row in ROWS]  # dye in col 1

# set pipetting depth to 1 mm below surface to avoid splash/film
dye_lld = {**lld_surface, "immersion_depth": [1]*8}

dye_tests = [
    (range(1, 5),   10),   # cols 1-4  → 10 µL
    (range(5, 9),   50),   # cols 5-8  → 50 µL
    (range(9, 12), 100),   # cols 9-11 → 100 µL
]

for cols, vol in dye_tests:
    for col in cols:
        dst_wells = [first_well(dst_plate[f"{row}{col}"]) for row in ROWS]
        try:
            await lh.aspirate(dye_src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, **lld_surface_follow)
        except:
            print ("Channels woudl have errored here.")
            await lh.aspirate(dye_src_wells, vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[1]*8)
        # await lh.aspirate(dye_src_wells, vols=[vol]*8, use_channels=USE_CHANS)
        await lh.dispense(dst_wells,     vols=[vol]*8, use_channels=USE_CHANS, **precise, liquid_height=[5]*8)                      # no blow-out
        # await lh.dispense(dst_wells,     vols=[vol]*8, use_channels=USE_CHANS, liquid_height=[5]*8)                      # no blow-out

###############################################################################
# 7) final discard
###############################################################################
# await lh.dispense(dye_src_wells, vols=[0]*8, use_channels=USE_CHANS,
#                   blow_out=[1]*8)                         # full channel blow-out
await lh.discard_tips()

/home/hamilton-robot/Documents/Hamilton-Starlet/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:332: UserWarning: Extra arguments to backend.aspirate: {'aspirate_flow_rate', 'dispense_flow_rate', 'end_delay_seconds'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")
/home/hamilton-robot/Documents/Hamilton-Starlet/pylabrobot/pylabrobot/liquid_handling/liquid_handler.py:332: UserWarning: Extra arguments to backend.dispense: {'aspirate_flow_rate', 'dispense_flow_rate', 'end_delay_seconds'}
  warnings.warn(f"Extra arguments to backend.{method.__name__}: {extra}")


Channels woudl have errored here.


In [ ]:
# await lh.dispense(src, vols=[50]*8, use_channels=[0], blow_out=[1])
# await lh.blow_out(location=waste["A1"], use_channels=channels)

# tip_rack = lh.deck.get_resource("htf_tips")
# await lh.drop_tips(tip_rack["A10:H10"], use_channels=list(range(8)))
# await lh.drop_tips(tiprack["A1:C1"])
# await lh.discard_tips()
# await lh.prepare_for_manual_channel_operation()
# await lh.stop()